<a href="https://colab.research.google.com/github/ayannj13/Data-Science/blob/main/Homework7_AyanJamal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The definition of the variables used in the Bookbinder dataset is given below. Use choice variable as dependent variable.

Choice: Whether the customer purchased the The Art History of Florence. 1 corresponds to a purchase and 0 corresponds to a nonpurchase.

Gender: 0 = Female and 1 = Male.

Amount purchased: Total money spent on BBBC books.

Frequency: Total number of purchases in the chosen period (used as a proxy for frequency.)

Last purchase (recency of purchase): Months since last purchase.

First purchase: Months since first purchase.

P_Child: Number of children’s books purchased.

P_Youth: Number of youth books purchased.

P_Cook: Number of cookbooks purchased.

P_DIY: Number of do-it-yourself books purchased.

P_Art: Number of art books purchased.


In [1]:

import pandas as pd
import numpy as np
df = pd.read_excel('https://hogut.weebly.com/uploads/1/8/1/6/18163409/bookbinder2.xls')
df.head()

,Gender,Amount purchased,Frequency,Last purchase,First purchase,P_Child,P_Youth,P_Cook,P_DIY,P_Art,Staus,Choice
0,1,113,8,1,8,0,1,0,0,0,Training,Y
1,1,418,6,11,66,0,2,3,2,3,Training,Y
2,1,336,18,6,32,2,0,1,1,2,Training,Y
3,1,180,16,5,42,2,0,0,1,1,Training,Y
4,0,320,2,3,18,0,0,0,1,2,Training,Y


In [2]:
# Using dataframe df: convert choice to 0(N)and 1(Y)

# Convert 'Choice' column to numerical values (0 for 'N', 1 for 'Y')
df['Choice'] = df['Choice'].map({'N': 0, 'Y': 1})
 #Using dataframe df: Use choice variable as dependent variable.
 #Construct training data and test data  when the status variables having  Training and  Test value.

import pandas as pd
from sklearn.linear_model import LogisticRegression

# Assuming 'df' is your DataFrame
# Filter the DataFrame for training data
training_data = df[df['Staus'] == 'Training']
test_data = df[df['Staus'] == 'Test']
# Define features (X) and target variable (y)
X_train = training_data[['Gender', 'Amount purchased', 'Frequency', 'Last purchase', 'First purchase', 'P_Child', 'P_Youth', 'P_Cook', 'P_DIY', 'P_Art']]
y_train = training_data['Choice']
X_test = test_data[['Gender', 'Amount purchased', 'Frequency', 'Last purchase', 'First purchase', 'P_Child', 'P_Youth', 'P_Cook', 'P_DIY', 'P_Art']]
y_test = test_data['Choice']


In [3]:
#Q1(25 pt)-Using logistic regression model in train data,
#a)	In the model, test that the claim that the coefficients of independent variables are different than 0 at the 5% significance level.
#b)	Which variables that is significantly different than 0 increase or decrease the probability of Choice=Yes?

import statsmodels.api as sm

# Add constant (intercept) manually
X_train_sm = sm.add_constant(X_train)

# Fit logistic regression model using statsmodels
logit_model = sm.Logit(y_train, X_train_sm)
result = logit_model.fit()

# Show summary with p-values and coefficients
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.435050
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 Choice   No. Observations:                 1600
Model:                          Logit   Df Residuals:                     1589
Method:                           MLE   Df Model:                           10
Date:                Mon, 14 Apr 2025   Pseudo R-squ.:                  0.2264
Time:                        12:07:23   Log-Likelihood:                -696.08
converged:                       True   LL-Null:                       -899.74
Covariance Type:            nonrobust   LLR p-value:                 2.612e-81
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.3515      0.214     -1.640      0.101      -0.772       0.069
Gender     

**Q1**

**a)** **Testing if coefficients are significantly different from zero at 5% level:**

To test whether the coefficients of the independent variables are different from 0 at the 5% significance level, we used logistic regression and checked the p-values (P>|z|) of each variable. If the p-value is less than 0.05, the variable is considered significantly different from 0.

Based on the regression output:

The variables **Gender, Amount purchased, Frequency, Last purchase, P_Child, P_Youth, P_Cook, P_DIY, and P_Art** are significantly different from 0 because their p-values are less than 0.05.

The variable First purchase is not significantly different from 0, as its p-value is greater than 0.05.

**b)** **Interpretation of how these significant variables affect the probability of purchase (Choice = Yes):**

To determine whether the significant variables increase or decrease the probability of purchase (Choice = Yes), we look at the **sign of their coefficients.**

A **positive** coefficient means the variable **increases** the probability of purchase.

A **negative** coefficient means the variable **decreases** the probability of purchase.

According to the model:

The variables **Amount purchased, Last purchase, and P_Art **have positive coefficients, so they increase the probability of purchase.

The variables **Gender, Frequency, P_Child, P_Youth, P_Cook, and P_DIY** have negative coefficients, so they decrease the probability of purchase.

The variable First purchase is not significant, so we do not interpret its effect.


In [4]:
#Q2(15 pt)-Using knn model in train data(choose k based on CV),
#get the performance metric on test data(accuracy,recall, precision confusion matrix). What is the k value chosen based on k?

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
import numpy as np

# Search for best k using 5-fold cross-validation on training data
k_range = range(1, 31)
cv_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

# Find the best k (with highest average CV accuracy)
best_k = k_range[np.argmax(cv_scores)]
print("Best k value:", best_k)

# Train KNN model using best_k
knn_model = KNeighborsClassifier(n_neighbors=best_k)
knn_model.fit(X_train, y_train)

# Predict on test data
y_pred = knn_model.predict(X_test)

# Evaluate metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:\n", conf_matrix)

Best k value: 29
Accuracy: 0.9026086956521739
Recall: 0.03431372549019608
Precision: 0.20588235294117646
Confusion Matrix:
 [[2069   27]
 [ 197    7]]


**Q2:**

To build the KNN model, we first selected the optimal value of k using 5-fold cross-validation on the training data. The value of k that resulted in the highest average cross-validation accuracy was 29.

Using this best k = 29, we trained the KNN model and evaluated its performance on the test data. From the performance metrics we got, from confusion matrix we see that the model correctly predicted 2069 non-purchasers and 7 purchasers. However, it missed 197 actual purchasers (false negatives), and it incorrectly predicted 27 non-purchasers as purchasers (false positives). The **recall is quite low**, indicating that the model struggles to identify positive cases (Choice = Yes), despite having high overall accuracy.


In [10]:
#Q3(35 pt)-Get the performance metric on test data (accuracy,recall, precision, confsuion matrix area under curve(auc)) using
#Logistic regression,naive bayes, Multi layer perceptor(MLP) and SVM, ligtGBM and kNN with optimized parameter in Q2.
#Report which methods performs best on these metrics?

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, roc_auc_score

# Dictionary to store results
results = {}

# 1. Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
y_proba_lr = lr.predict_proba(X_test)[:, 1]

results['Logistic Regression'] = {
    'accuracy': accuracy_score(y_test, y_pred_lr),
    'recall': recall_score(y_test, y_pred_lr),
    'precision': precision_score(y_test, y_pred_lr),
    'confusion_matrix': confusion_matrix(y_test, y_pred_lr),
    'auc': roc_auc_score(y_test, y_proba_lr)
}

# 2. Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
y_proba_nb = nb.predict_proba(X_test)[:, 1]

results['Naive Bayes'] = {
    'accuracy': accuracy_score(y_test, y_pred_nb),
    'recall': recall_score(y_test, y_pred_nb),
    'precision': precision_score(y_test, y_pred_nb),
    'confusion_matrix': confusion_matrix(y_test, y_pred_nb),
    'auc': roc_auc_score(y_test, y_proba_nb)
}

# 3. Multi-Layer Perceptron
mlp = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=1))
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)
y_proba_mlp = mlp.predict_proba(X_test)[:, 1]

results['MLP'] = {
    'accuracy': accuracy_score(y_test, y_pred_mlp),
    'recall': recall_score(y_test, y_pred_mlp),
    'precision': precision_score(y_test, y_pred_mlp),
    'confusion_matrix': confusion_matrix(y_test, y_pred_mlp),
    'auc': roc_auc_score(y_test, y_proba_mlp)
}

# 4. Support Vector Machine (SVM with probability)
svm = make_pipeline(StandardScaler(), SVC(probability=True))
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
y_proba_svm = svm.predict_proba(X_test)[:, 1]

results['SVM'] = {
    'accuracy': accuracy_score(y_test, y_pred_svm),
    'recall': recall_score(y_test, y_pred_svm),
    'precision': precision_score(y_test, y_pred_svm),
    'confusion_matrix': confusion_matrix(y_test, y_pred_svm),
    'auc': roc_auc_score(y_test, y_proba_svm)
}

# 5. LightGBM
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)
y_proba_lgbm = lgbm.predict_proba(X_test)[:, 1]

results['LightGBM'] = {
    'accuracy': accuracy_score(y_test, y_pred_lgbm),
    'recall': recall_score(y_test, y_pred_lgbm),
    'precision': precision_score(y_test, y_pred_lgbm),
    'confusion_matrix': confusion_matrix(y_test, y_pred_lgbm),
    'auc': roc_auc_score(y_test, y_proba_lgbm)
}

# 6. KNN (with best_k = 29 from Q2)
knn = KNeighborsClassifier(n_neighbors=29)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
y_proba_knn = knn.predict_proba(X_test)[:, 1]

results['KNN (k=29)'] = {
    'accuracy': accuracy_score(y_test, y_pred_knn),
    'recall': recall_score(y_test, y_pred_knn),
    'precision': precision_score(y_test, y_pred_knn),
    'confusion_matrix': confusion_matrix(y_test, y_pred_knn),
    'auc': roc_auc_score(y_test, y_proba_knn)
}

# Print all model results
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    print("Confusion Matrix:")
    print(metrics['confusion_matrix'])


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 400, number of negative: 1200
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000257 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 361
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.250000 -> initscore=-1.098612
[LightGBM] [Info] Start training from score -1.098612

Model: Logistic Regression
Accuracy: 0.8896
Recall: 0.3775
Precision: 0.3775
AUC: 0.8003
Confusion Matrix:
[[1969  127]
 [ 127   77]]

Model: Naive Bayes
Accuracy: 0.8483
Recall: 0.3088
Precision: 0.2325
AUC: 0.7444
Confusion Matrix:
[[1888  208]
 [ 141   63]]

Model: MLP
Accuracy: 0.8639
Recall: 0.4118
Precision: 0.3032
AUC: 0.7841
Confusion Matrix:
[[1903  193]


**Q3:**

Using the test data, the models gave the following best performance metrics:

**Best accuracy: **KNN and SVM (0.9026)

**Best recall:** MLP (0.4118)

**Best precision:** SVM (0.4180)

**Best AUC:** Logistic Regression (0.8003)

Although KNN had the highest accuracy, it performed poorly on recall and AUC. Overall, SVM performed best in terms of accuracy and precision, and Logistic Regression was best for AUC. MLP had the best recall, meaning it identified more actual buyers.


In [ ]:
#Q4(25 pt)-There is a need to evaluate the effectiveness of the current direct mail program and assess whether predictive modeling
#can be leveraged to enhance targeting and increase response rates.
# Assume that you make profit of 10 $ by selling a book and the cost of sending mail is 0.65 $.
# Based on the logistic model, which customers should Bookbinders target on test data?
#What is the expected profits if these customers are targeted?
#Hint :
#To evaluate the direct mail program, use an ROC curve to assess the logistic model’s ability to predict buyers.
#The curve plots True Positive Rate vs. False Positive Rate, and a high AUC shows the model is effective.
# To decide who to target, calculate profit for different probability thresholds:
# Profit = (True Positives × $9.35) - (False Positives × $0.65).
#True positive profit is $93.5 becuase revenue is $10 and there is a $0.65 cost of sending mail.
#Bookbinders should target customers with probabilities above the threshold that maximizes profit,
# improving targeting and response rates compared to the current program


In [11]:
#Q4
import numpy as np
from sklearn.metrics import confusion_matrix

thresholds = np.arange(0.01, 1.00, 0.01)
profits = []

for t in thresholds:
    y_pred_thresh = (y_proba_lr >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thresh).ravel()
    profit = (tp * 9.35) - (fp * 0.65)
    profits.append(profit)

# Find the best threshold
best_index = np.argmax(profits)
best_threshold = thresholds[best_index]
max_profit = profits[best_index]

print("Best threshold:", best_threshold)
print("Maximum expected profit:", max_profit)


Best threshold: 0.17
Maximum expected profit: 1047.4


**Q4:**

To evaluate the effectiveness of the current direct mail program and improve targeting, we used the logistic regression model trained earlier to generate predicted probabilities for each customer in the test data.

Using these probabilities, we assessed the model’s performance using an ROC-based approach. We tested different probability thresholds and calculated expected profit at each threshold using the formula:

Profit = (True Positives × 9.35) − (False Positives × 0.65)
After iterating through thresholds from 0.01 to 0.99, the threshold that maximizes profit was found to be 0.17. This means Bookbinders should target customers whose predicted probability of purchasing is greater than or equal to 0.17.

At this threshold, the maximum expected profit is $1047.40.

This approach shows that predictive modeling significantly improves targeting and allows Bookbinders to increase profits by sending mail only to customers most likely to purchase, rather than mailing everyone.